In [ ]:
import pandas as pd
import numpy as np
import importlib
import tTsTGrpUtils as tsutil

In [ ]:
dl_pth = "/host/verges/tank/data/daniel/3T7T/z/outputs/05b_stats_winStudy_grp_18Sep2025-125541.pkl"
dl = tsutil.loadPickle(dl_pth)
tsutil.print_dict(dl)

In [ ]:
importlib.reload(tsutil)
dl_out = tsutil.parcellate_items(dl=dl, df_key_in='df_w_TLE_L', parc='glasser', region = 'cortex', parc_lbl='glasser_name_short', verbose=True)

In [ ]:
item = dl_out[15]
item.keys()

In [ ]:
df = item['df_w_TLE_L_parc-glsr']
cols = df.columns
print(f"Shape of df: {df.shape}")
print(list(cols))
print(f"N unique: {len(np.unique(cols))}")


In [ ]:
importlib.reload(tsutil)
tsutil.print_dict(dl_out, df_print=True, idx=15)

In [ ]:
importlib.reload(tsutil)

df_parcel = tsutil.apply_glasser(df, surf=surf, 
                          labelType='glasser_name_short', addHemiLbl=True, 
                          ipsiTo='L')
print(f"df_parcel shape: {df_parcel.shape}")
print(f"df_parcel columns (first 10): {df_parcel.columns[:10]}")

In [ ]:
df.columns

In [ ]:
glasser_details = pd.read_csv("/host/verges/tank/data/daniel/parcellations/glasser_details.csv")

In [ ]:
glasser_details['regionLongName']